In [21]:
from langchain_core.documents import Document

In [22]:
documents=[
    Document(
        page_content="Dogs are greate companions, known for their loyalty and friendliness.",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners,repairing relatively simple care.",
        metadata={"source":"fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source":"bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source":"mammal-pets-doc"},
    )
]

In [23]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are greate companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners,repairing relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [31]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(api_key=groq_api_key,model="llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000023D28603740>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023D286034D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings=OllamaEmbeddings(model="gemma:2b")

In [26]:
## Vector Store
from langchain_community.vectorstores import FAISS

vectorstore=FAISS.from_documents(documents,embeddings)
vectorstore

In [27]:
vectorstore.similarity_search("cat")

[Document(id='01e19106-aeb8-4745-af99-20038979a2a6', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are greate companions, known for their loyalty and friendliness.'),
 Document(id='7e682603-be91-4f64-9286-5d6c93af91e4', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='0c6ba551-3d39-4e52-aa5c-884437e488c9', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners,repairing relatively simple care.'),
 Document(id='06fbd2da-f2c1-407b-806e-94930a879ed4', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [28]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='01e19106-aeb8-4745-af99-20038979a2a6', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are greate companions, known for their loyalty and friendliness.'),
  np.float32(4638.9023)),
 (Document(id='7e682603-be91-4f64-9286-5d6c93af91e4', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  np.float32(4681.8)),
 (Document(id='0c6ba551-3d39-4e52-aa5c-884437e488c9', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners,repairing relatively simple care.'),
  np.float32(4946.6484)),
 (Document(id='06fbd2da-f2c1-407b-806e-94930a879ed4', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  np.float32(4958.92))]

### Retrivers

In [29]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='01e19106-aeb8-4745-af99-20038979a2a6', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are greate companions, known for their loyalty and friendliness.')],
 [Document(id='01e19106-aeb8-4745-af99-20038979a2a6', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are greate companions, known for their loyalty and friendliness.')]]

In [ ]:
##RAG
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message="""
Answer this question using the provided context only.
{question}
Context:
{context}
"""
prompt=ChatPromptTemplate.from_messages([("human",message)])
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("tell me about dogs")
print(response)

content='Dogs are great companions, known for their loyalty and friendliness.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 108, 'total_tokens': 123, 'completion_time': 0.01844556, 'completion_tokens_details': None, 'prompt_time': 0.008150307, 'prompt_tokens_details': None, 'queue_time': 0.056122283, 'total_time': 0.026595867}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--ef0262dd-e965-4834-b8f3-d7c4405e16d6-0' usage_metadata={'input_tokens': 108, 'output_tokens': 15, 'total_tokens': 123}
